## Computing PCA using RDDs

##  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We will perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### The effect of  `nan`s in arithmetic operations
* We use an RDD of numpy arrays, instead of Dataframes.
* Why? Because numpy treats `nan` entries correctly:
  * In numpy `5+nan=5` while in dataframes `5+nan=nan`

### Performing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

import numpy as np
X=np.array([1,1,1,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)
X=np.array([1,1,np.NaN,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)

#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

x1=np.array([1,np.NaN,3,4,5])
x2=np.array([2,3,4,np.NaN,6])
stacked=np.array([np.outer(x1,x1),np.outer(x2,x2)])
stacked

np.nanmean(stacked,axis=0)

In [28]:
import findspark
findspark.init()
from pyspark import SparkContext

sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark.sql import *
sqlContext = SQLContext(sc)

In [29]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into 256 geographical rectangles, indexed from BBBBBBBB to SSSSSSSS. And each containing about 12,000 station,year pairs.

In [2]:
#file_index='BBBSBBBB'
file_index='SSSBBSSB'
data_dir='../../Data/Weather'

filebase='US_Weather_%s'%file_index

In [ ]:

!rm -rf $data_dir/$filebase*

In [5]:
c_filename=filebase+'.csv.gz'
u_filename=filebase+'.csv'

In [6]:

command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(c_filename,data_dir,c_filename)
print command
!$command


curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_SSSBBSSB.csv.gz > ../../Data/Weather/US_Weather_SSSBBSSB.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3302k  100 3302k    0     0  1423k      0  0:00:02  0:00:02 --:--:-- 1480k


In [7]:
!ls $data_dir/$c_filename

../../Data/Weather/US_Weather_SSSBBSSB.csv.gz


In [8]:
!ls -lh $data_dir/$c_filename

-rw-r--r--  1 toby  staff   3.2M May 15 20:20 ../../Data/Weather/US_Weather_SSSBBSSB.csv.gz


In [6]:
#unzip
!gunzip -c $data_dir/$c_filename > $data_dir/$u_filename

In [30]:
import pickle
List=pickle.load(open(data_dir+'/'+u_filename,'rb'))
len(List)

12433

In [31]:
df=sqlContext.createDataFrame(List)
print df.count()
df.show(6)

12433
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|   1731.9| 37.6833|-113.0833|       TMAX|USC00421272|     4|[80 4D 80 C9 00 0...|1907.0|SSSBBSSB|
|   1731.9| 37.6833|-113.0833|       TMAX|USC00421272|    29|[00 7E A0 53 00 C...|1911.0|SSSBBSSB|
|   1731.9| 37.6833|-113.0833|       TMAX|USC00421272|    18|[40 CC 40 4C 80 C...|1912.0|SSSBBSSB|
|   1731.9| 37.6833|-113.0833|       TMAX|USC00421272|    36|[40 56 A0 56 58 5...|1915.0|SSSBBSSB|
|   1731.9| 37.6833|-113.0833|       TMAX|USC00421272|    40|[20 50 30 54 30 5...|1916.0|SSSBBSSB|
|   1731.9| 37.6833|-113.0833|       TMAX|USC00421272|    38|[20 50 00 46 80 4...|1917.0|SSSBBSSB|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only

In [11]:
#store dataframe as parquet file
outfilename=data_dir+'/'+filebase+'.parquet'
!rm -rf $outfilename
df.write.save(outfilename)

In [12]:
# Compare file sizes
!du -sh $data_dir/$filebase*

 12M	../../Data/Weather/US_Weather_SSSBBSSB.csv
3.2M	../../Data/Weather/US_Weather_SSSBBSSB.csv.gz
4.0M	../../Data/Weather/US_Weather_SSSBBSSB.parquet


In [13]:
from time import time
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
print 'took',time()-t,'seconds'

Number of executors= 3
took 0.00145602226257 seconds


In [14]:
measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']

In [32]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [33]:
Query="SELECT * FROM weather\n\tWHERE measurement = 'SNWD'"
print Query
df = sqlContext.sql(Query)
alldf = df.toPandas()

SELECT * FROM weather
	WHERE measurement = 'SNWD'


37.272199999999998, 38.0, -115.16670000000001, -109.08280000000001)

In [34]:
Query="""SELECT station, elevation, latitude, longitude 
FROM weather\n\tWHERE
GROUP BY 
station,
elevation,
latitude, longitude
"""
print Query
df = sqlContext.sql(Query)
groupstation = df.toPandas()
groupstation

SELECT station, elevation, latitude, longitude 
FROM weather
	WHERE
GROUP BY 
station,
elevation,
latitude, longitude



,station,elevation,latitude,longitude
0,USS0013M05S,2819.4,37.5333,-113.0500
1,USS0012M05S,2377.4,37.4833,-112.5833
2,USC00422561,1569.7,37.7697,-113.6556
3,USC00427501,1587.7,38.0000,-113.4500
4,USC00260015,1684.0,37.5167,-114.1667
5,USS0012M23S,2987.0,37.5667,-112.8333
6,USR0000ASSA,2468.9,37.5167,-112.5556
7,USC00422597,1220.1,37.3167,-110.9000
8,USR0000ENTR,1627.6,37.5586,-113.7172
9,USC00423847,1833.1,37.5667,-112.0000


In [35]:
groupstation["station"].count()

99

In [36]:
Query="""SELECT station, year, vector 
FROM weather\n\tWHERE measurement = 'SNWD'
"""
print Query
df = sqlContext.sql(Query)
snwd = df.toPandas()
snwd

SELECT station, year, vector 
FROM weather
	WHERE measurement = 'SNWD'



,station,year,vector
0,USC00421308,1961.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,USC00421308,1966.0,"[0, 126, 0, 126, 0, 126, 0, 126, 0, 126, 0, 12..."
2,USC00421308,1967.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,USC00421308,1968.0,"[196, 92, 196, 92, 196, 92, 196, 92, 196, 92, ..."
4,USC00421308,1969.0,"[88, 90, 144, 89, 144, 89, 144, 89, 144, 89, 1..."
5,USC00421308,1971.0,"[0, 0, 0, 0, 240, 91, 240, 91, 240, 91, 240, 9..."
6,USC00421308,1972.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,USC00421308,1975.0,"[0, 126, 240, 87, 240, 87, 240, 87, 240, 87, 2..."
8,USC00421308,1976.0,"[192, 88, 192, 88, 192, 88, 192, 88, 192, 88, ..."
9,USC00421308,1977.0,"[64, 78, 0, 126, 88, 90, 88, 90, 92, 92, 240, ..."


In [25]:
def c2f(c):
    return 9.0/5.0 * c + 32

def proc(t):
    year = t[0]
    x = t[1]
    x[np.isnan(x)] = 0
    return year, np.max(x) / 10

def maxval(t1, t2):
    max = t1[1]
    year_max = t1[0]
    x1 = t1[1]
    x2 = t2[1]
    
    if x1 >= x2:
        max = x1
        year_max = t1[0]
    else:
        max = x2
        year_max = t2[0]
        
    return year_max, max

meas='TMAX'
Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
print Query
df = sqlContext.sql(Query)
data=df.rdd.map(lambda row: (row['year'], unpackArray(row['vector'],np.float16)))
data_filtered = data.map(proc)
print data_filtered.take(10)
maxvalrd = data_filtered.reduce(maxval)
print "Highest recorded temperature is {0} deg F in {1}".format(c2f(maxvalrd[1]), maxvalrd[0])

SELECT * FROM weather
	WHERE measurement = 'TMAX'


AnalysisException: u'Table or view not found: weather; line 1 pos 14'

In [37]:
def proc(t):
    year = t[0]
    x = t[1]
    x[np.isnan(x)] = 0
    return year, np.min(x) / 10

def minval(t1, t2):
    max = t1[1]
    year_min = t1[0]
    x1 = t1[1]
    x2 = t2[1]
    
    if x1 <= x2:
        max = x1
        year_min = t1[0]
    else:
        max = x2
        year_min = t2[0]
        
    return year_min, max

meas='TMIN'
Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
print Query
df = sqlContext.sql(Query)
data=df.rdd.map(lambda row: (row['year'], unpackArray(row['vector'],np.float16)))
data_filtered = data.map(proc)
print data_filtered.take(10)
minvalrd = data_filtered.reduce(minval)
print minvalrd
print "Lowest recorded temperature is {0} deg F in {1}".format(c2f(minvalrd[1]), minvalrd[0])

SELECT * FROM weather
	WHERE measurement = 'TMIN'
[(1906.0, -16.100000000000001), (1907.0, -14.4), (1911.0, -18.899999999999999), (1912.0, -18.300000000000001), (1915.0, -20.600000000000001), (1917.0, -23.300000000000001), (1922.0, -23.899999999999999), (1923.0, -20.600000000000001), (1924.0, -18.899999999999999), (1925.0, -18.300000000000001)]
(1942.0, -25.0)
Lowest recorded temperature is -13.0 deg F in 1942.0


In [211]:
def proc(t):
    year = t[0]
    x = t[1]
    x[np.isnan(x)] = 0
    return year, np.max(x) / 10

def maxval(t1, t2):
    max = t1[1]
    year_max = t1[0]
    x1 = t1[1]
    x2 = t2[1]
    
    if x1 >= x2:
        max = x1
        year_max = t1[0]
    else:
        max = x2
        year_max = t2[0]
        
    return year_max, max

meas='SNWD'
Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
print Query
df = sqlContext.sql(Query)
data=df.rdd.map(lambda row: (row['year'], unpackArray(row['vector'],np.float16)))
data_filtered = data.map(proc)
print data_filtered.take(10)
maxvalrd = data_filtered.reduce(maxval)
print "Strongest recorded snow activity is in {1} where snow-depth reached {0} inches".format(c2f(maxvalrd[1]), maxvalrd[0])

SELECT * FROM weather
	WHERE measurement = 'SNWD'
[(1961.0, 22.899999999999999), (1966.0, 22.899999999999999), (1967.0, 76.200000000000003), (1968.0, 30.5), (1969.0, 40.600000000000001), (1971.0, 50.799999999999997), (1972.0, 55.899999999999999), (1975.0, 27.899999999999999), (1976.0, 25.399999999999999), (1977.0, 30.5)]
Strongest recorded snow activity is in 1970.0 where snow-depth reached 342.86 inches


In [38]:
def proc(t):
    year = t[0]
    x = t[1]
    x[np.isnan(x)] = 0
    return year, np.max(x) / 10

def maxval(t1, t2):
    max = t1[1]
    year_max = t1[0]
    x1 = t1[1]
    x2 = t2[1]
    
    if x1 >= x2:
        max = x1
        year_max = t1[0]
    else:
        max = x2
        year_max = t2[0]
        
    return year_max, max

meas='PRCP'
Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
print Query
df = sqlContext.sql(Query)
data=df.rdd.map(lambda row: (row['year'], unpackArray(row['vector'],np.float16)))
data_filtered = data.map(proc)
print data_filtered.take(10)
maxvalrd = data_filtered.reduce(maxval)
print "Strongest recorded rainfall is in {1} with staggering {0} inches".format(c2f(maxvalrd[1]), maxvalrd[0])

SELECT * FROM weather
	WHERE measurement = 'PRCP'
[(1950.0, 13.199999999999999), (1906.0, 37.799999999999997), (1907.0, 40.600000000000001), (1908.0, 44.200000000000003), (1912.0, 52.100000000000001), (1913.0, 20.300000000000001), (1916.0, 68.599999999999994), (1917.0, 39.399999999999999), (1919.0, 21.600000000000001), (1928.0, 30.5)]
Strongest recorded rainfall is in 2006.0 with staggering 166.82 inches


In [39]:
def proc(t):
    year = t[0]
    x = t[1]
    x[np.isnan(x)] = 0
    return year, x

def maxval(t1, t2):
    max = t1[1]
    year_max = t1[0]
    x1 = t1[1]
    x2 = t2[1]
    
    if x1 >= x2:
        max = x1
        year_max = t1[0]
    else:
        max = x2
        year_max = t2[0]
        
    return year_max, max

meas='SNWD'
Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
print Query
df = sqlContext.sql(Query)
data=df.rdd.map(lambda row: (row['year'], unpackArray(row['vector'],np.float16)))
data_filtered = data.map(proc)
print data_filtered.take(10)
maxvalrd = data_filtered.reduce(maxval)
print "Strongest recorded snow activity is in {1} where snow-depth reached {0} inches".format(c2f(maxvalrd[1]), maxvalrd[0])

SELECT * FROM weather
	WHERE measurement = 'SNWD'
[(1961.0, array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  152.,
          0.,    0.,    0.,  127.,  127.,  127.,  102.,  102.,  102.,
        102.,   76.,   51.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,   76.,  152.,
        178.,  229.,    0.,  127.,   51.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,   25.,   51.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,   

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 14.0 failed 1 times, most recent failure: Lost task 2.0 in stage 14.0 (TID 229, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/toby/s/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/toby/s/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/toby/s/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/toby/s/spark-2.1.0-bin-hadoop2.7/python/pyspark/rdd.py", line 833, in func
    yield reduce(f, iterator, initial)
  File "<ipython-input-

In [40]:
from numpy import linalg as LA
STAT={}  # dictionary storing the statistics for each measurement
Clean_Tables={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
    print Query
    df = sqlContext.sql(Query)
    data=df.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
    #get very basic statistics
    STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

    # compute covariance matrix
    OUT=computeCov(data)

    #find PCA decomposition
    eigval,eigvec=LA.eig(OUT['Cov'])

    # collect all of the statistics in STAT[meas]
    STAT[meas]['eigval']=eigval
    STAT[meas]['eigvec']=eigvec
    STAT[meas].update(OUT)

    print 'time for',meas,'is',time()-t

SELECT * FROM weather
	WHERE measurement = 'TMAX'
shape of E= (365,) shape of NE= (365,)
time for TMAX is 20.4163279533
SELECT * FROM weather
	WHERE measurement = 'SNOW'
shape of E= (365,) shape of NE= (365,)
time for SNOW is 17.1124579906
SELECT * FROM weather
	WHERE measurement = 'SNWD'
shape of E= (365,) shape of NE= (365,)
time for SNWD is 13.2979488373
SELECT * FROM weather
	WHERE measurement = 'TMIN'
shape of E= (365,) shape of NE= (365,)
time for TMIN is 18.9574279785
SELECT * FROM weather
	WHERE measurement = 'PRCP'
shape of E= (365,) shape of NE= (365,)
time for PRCP is 21.5000188351
SELECT * FROM weather
	WHERE measurement = 'TOBS'
shape of E= (365,) shape of NE= (365,)
time for TOBS is 12.8827271461


In [41]:
STAT.keys()

['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP']

In [42]:
STAT['TMAX']['eigvec'].shape

(365, 365)

In [43]:
from pickle import dump
filename=data_dir+'/STAT_%s.pickle'%file_index
dump((STAT,STAT_Descriptions),open(filename,'wb'))
